<a href="https://colab.research.google.com/github/scissorsgh/legend_calc/blob/main/%ED%95%A8%EC%88%98%EB%A7%9E%EC%B6%94%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np

def sin_dataset(num_points):
    x_min, x_max = -2*np.pi, 2*np.pi # 점 생성 범위 지정
    x = np.linspace(x_min, x_max, num_points) # 범위 내에서 num_points 개의 점을 균등한 간격으로 생성
    y = np.sin(x) # sin(x)의 값으로 y 생성
    return x, y

def cos_dataset(num_points):
    x_min, x_max = -2*np.pi, 2*np.pi
    x = np.linspace(x_min, x_max, num_points)
    y = np.cos(x) # cos(x)의 값으로 y 생성
    return x, y

def tan_dataset(num_points):
    x_min, x_max = -1.4, 1.4 # 불연속이라서 안정된 범위를 지정
    x = np.linspace(x_min, x_max, num_points)
    y = np.clip(np.tan(x), -3, 3) # tan(x)의 값으로 y 생성
    return x, y

def log_dataset(num_points):
    x_min, x_max = 0.1, 5
    x = np.linspace(x_min, x_max, num_points)
    y = np.log(x) # log x의 값으로 y 생성
    return x, y

def exp_dataset(num_points):
    x_min, x_max = -1.5, 1.5
    x = np.linspace(x_min, x_max, num_points)
    y = np.exp(x) # a^x의 값으로 y 생성
    return x, y

def generate_dataset(num_points):
    X = []
    y = []
    func_map = [sin_dataset, cos_dataset, tan_dataset, log_dataset, exp_dataset]

    for i in range(5):
        func = func_map[i]
        for _ in range(num_points): #데이터셋 만들기
            x, y_vals = func(100) # 점 100개 생성
            noise = np.random.normal(0, 0.1, 100)
            y_noisy = y_vals + noise
            coords = np.column_stack([x, y_noisy]) # x, y_noisy가 한 쌍의 좌표로 묶여 (x, y) 형식
            X.append(coords)
            y.append(i) # 넘버링 (y는 3차원 배열이 됨)
    return X, y

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__ (self, X, y):
    self.X = torch.FloatTensor(X) # (x, y)쌍 (실수형)
    self.y = torch.LongTensor(y) # 넘버링 숫자 (정수형)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, idx):
    coords = self.X[idx]
    flattened = coords.flatten()
    return torch.FloatTensor(flattened), torch.LongTensor([self.y[idx]])[0]

In [26]:
X_train, y_train = generate_dataset(25) # 학습용 25개 만들기
X_val, y_val = generate_dataset(2) #테스트용 2개 만들기

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) # 학습용 데이터 불러오기
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True) # 테스트용 데이터 불러오기

In [4]:
import torch.nn as nn

class FunctionClassifier(nn.Module): # 학습하기 위한 모델 만들기
  def __init__(self, input_size=200, num_classes=5): # x축 점 100개, y축 점 100개 크기로 초기화
    super(FunctionClassifier, self).__init__()
    self.network = nn.Sequential(
        nn.Linear(200,512), # 200 -> 512 데이터 확장
        nn.ReLU(),
        nn.Dropout(0.4), # 정규화 진행(편향방지) -> 퍼셉트론 40% 끄기
        nn.Linear(512, 256), # 데이터 최소화 (종류 분류를 위해)
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64, 5) # 5가지 특징 추출
    )

  def forward(self, x):
      return self.network(x)

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 사용 장치 정하기
model = FunctionClassifier().to(device)
criterion = nn.CrossEntropyLoss() # 손실 계산
optimizer = optim.Adam(model.parameters(), lr=0.001) # 최적화 하기

train_losses = [] # 학습중 손실
val_accuracies = [] # 테스트 정확도

num_epochs = 100 # 학습수
print("학습 시작...")

for epoch in range(num_epochs):
  model.train() # 학습 모드
  train_loss = 0.0 # 초기화
  for batch_X, batch_y in train_loader: # 학습 하기
    batch_X = batch_X.view(batch_X.size(0), -1) #자동 사이즈 통일
    batch_X = batch_X.to(device)
    batch_y = batch_y.to(device)

    optimizer.zero_grad() # 기울기 초기화
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    loss.backward() # 역전파
    optimizer.step() # 새 가중치로 수정
    train_loss += loss.item() # 손실 합
  avg_train_loss = train_loss / len(train_loader) # 손실 평균 구하기
  train_losses.append(avg_train_loss)
  if (epoch+1) % 10 == 0:
    model.eval() # 평가 모드
    val_correct = 0 # 초기화
    val_total = 0

    with torch.no_grad(): # 기울기 계산 안함
      for batch_X, batch_y in val_loader: # 테스트 하기
        batch_X.view(batch_X.size(0), -1) #자동 사이즈 통일
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1) # 일치하는 함수 숫자 추출
        val_total += batch_y.size(0) # 맞춘 샘플 수
        val_correct += (predicted == batch_y).sum().item() # 정확도 구하기

    val_accuracy = 100*val_correct / val_total #평가 결과 정확도
    val_accuracies.append(val_accuracy)

    print(f'Epoch: [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

print("학습 완료!")

학습 시작...
Epoch: [10/100], Loss: 0.0025, Val Acc: 100.00%
Epoch: [20/100], Loss: 0.0009, Val Acc: 100.00%
Epoch: [30/100], Loss: 0.0010, Val Acc: 100.00%
Epoch: [40/100], Loss: 0.0000, Val Acc: 100.00%
Epoch: [50/100], Loss: 0.0000, Val Acc: 100.00%
Epoch: [60/100], Loss: 0.0001, Val Acc: 100.00%
Epoch: [70/100], Loss: 0.0000, Val Acc: 100.00%
Epoch: [80/100], Loss: 0.0000, Val Acc: 100.00%
Epoch: [90/100], Loss: 0.0000, Val Acc: 100.00%
Epoch: [100/100], Loss: 0.0000, Val Acc: 100.00%
학습 완료!


In [31]:
model.eval() # 평가 모드

test_functions = [sin_dataset, cos_dataset, tan_dataset, log_dataset, exp_dataset]
func_names = ['sin', 'cos', 'tan', 'log', 'exp']

for true_idx, (func, name) in enumerate(zip(test_functions, func_names)): # 카운트도 해주기, 라벨링

  # 노이즈 있는 데이터 생성
  x, y = func(100)
  noise = np.random.normal(0, 0.2, 100)
  y_noisy = y + noise
  coords = np.column_stack([x, y_noisy])

  # 예측
  test_input = torch.FloatTensor(coords.flatten()).unsqueeze(0).to(device)

  with torch.no_grad(): # 기울기 계산 안함
    output = model(test_input)
    _, predicted = torch.max(output, 1)
    predicted_idx = predicted.item()

  predicted_name = func_names[predicted_idx]
  result = "✓" if predicted_idx == true_idx else "✗" # 최종 평가 결과 출력

  print(f"{name} → {predicted_name} {result}")

sin → sin ✓
cos → cos ✓
tan → tan ✓
log → log ✓
exp → exp ✓
